In [ ]:
'''
    Author :  Afshin Karimi
    Sharif University of Technology
    Predict House Prices with Gaussian Kernel Regression
'''
import numpy as np
import pandas as pd
import plotly.express as px


In [ ]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv") 
train_copy = train_data.copy()
test_copy = test_data.copy()
# Dropping the sparse columns, ID (from both Trainig and test)
train_copy.drop(["Id","Alley","FireplaceQu","PoolQC","Fence","MiscFeature"], axis = 1, inplace=True)
test_copy.drop(["Id","Alley","FireplaceQu","PoolQC","Fence","MiscFeature"], axis = 1, inplace=True)
train_copy.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
# Missing value treatment. Replacing String Columns with 'NAs' and Numerical Columns with '-99' (junk value)

#Training data
columns_train = list(train_copy)
for i in columns_train:
    if train_copy[i].dtype == 'object':
        train_copy[i] = train_copy[i].fillna('Na')
    else:
        train_copy[i] = train_copy[i].fillna(-99)

In [ ]:
# Missing value treatment. Replacing String Columns with 'NAs' and Numerical Columns with '-99' (junk value)

# Test data
columns_test = list(test_copy)
for i in columns_test:
    if test_copy[i].dtype == 'object':
        test_copy[i] = test_copy[i].fillna('Na')
    else:
        test_copy[i] = test_copy[i].fillna(-99)

In [ ]:
# Seperating examples and labels from the training data set
X =  train_copy.drop(['SalePrice'], axis=1)
y = train_copy.SalePrice

In [ ]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal


In [ ]:
# Doing one hot encoding for both training and test data sets
X = pd.get_dummies(X)
test_copy = pd.get_dummies(test_copy)

In [ ]:
print("one hot encoded training data set shape", X.shape, "\n")
print("one hot encoded training data set shape", test_copy.shape, "\n")

one hot encoded training data set shape (1460, 281) 

one hot encoded training data set shape (1459, 271) 



In [ ]:
common_cols = [a for a in list(train_copy) for b in list(test_copy) if a==b]
len(common_cols)

36

In [ ]:
# Subsetting both train and test data sets for the common columns:
train_copy_comm = X[common_cols]
test_copy_comm = test_copy[common_cols]

In [ ]:
train_copy_comm.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008


In [ ]:
# Standardizing the training and test data set
mean = train_copy_comm.mean(axis = 0)
std = train_copy_comm.std(axis = 0)

train_copy_comm -= mean
train_copy_comm /= std

test_copy_comm -= mean
test_copy_comm /= std

In [ ]:
# The above operation has introduced NAs in the data set becuase of the division operation. We need to do the null value imputation again
train_copy_comm = train_copy_comm.fillna(0)
test_copy_comm = test_copy_comm.fillna(0)

In [ ]:
train_copy_comm.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.073350,0.365401,-0.207071,0.651256,-0.517023,1.050634,0.878367,0.515625,0.575228,-0.288554,-0.944267,-0.459145,-0.793162,1.161454,-0.120201,0.370207,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,0.911897,-0.950901,0.293390,0.311618,0.350880,-0.751918,0.216429,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-1.598563,0.138730
1,-0.872264,0.585178,-0.091855,-0.071812,2.178881,0.156680,-0.429430,-0.566122,1.171591,-0.288554,-0.641008,0.466305,0.257052,-0.794891,-0.120201,-0.482347,-0.819684,3.947457,0.789470,-0.761360,0.163723,-0.211381,-0.318574,0.600289,0.236708,0.311618,-0.060710,1.625638,-0.704242,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-0.488943,-0.614228
2,0.073350,0.409357,0.073455,0.651256,-0.517023,0.984415,0.829930,0.327975,0.092875,-0.288554,-0.301540,-0.313261,-0.627611,1.188943,-0.120201,0.514836,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,-0.318574,0.600289,0.289192,0.311618,0.631510,-0.751918,-0.070337,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,0.990552,0.138730
3,0.309753,0.292142,-0.096864,0.651256,-0.517023,-1.862993,-0.720051,-0.566122,-0.499103,-0.288554,-0.061648,-0.687089,-0.521555,0.936955,-0.120201,0.383528,1.107431,-0.240978,-1.025689,-0.761360,0.163723,-0.211381,0.296662,0.600289,0.282894,1.649742,0.790533,-0.751918,-0.175988,4.091122,-0.116299,-0.270116,-0.068668,-0.087658,-1.598563,-1.367186
4,0.073350,0.643785,0.375020,1.374324,-0.517023,0.951306,0.733056,1.365570,0.463410,-0.288554,-0.174805,0.199611,-0.045596,1.617323,-0.120201,1.298881,1.107431,-0.240978,0.789470,1.227165,1.389547,-0.211381,1.527133,0.600289,0.287092,1.649742,1.697903,0.779930,0.563567,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,2.100173,0.138730


In [371]:
# select 85 percent of train data for train set
X_train=train_copy_comm.sample(frac=0.85)
# select 15 percent of train data for test set
X_validation=train_copy_comm.drop(X_train.index)
Y_train=y.iloc[list(X_train.index.values.tolist())]
Y_train_validation=y.drop(Y_train.index)

In [358]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1259,-0.872264,0.365401,-0.076827,-0.071812,2.178881,-0.075086,-0.768488,-0.566122,0.347207,2.426559,-1.252052,-0.007817,-0.280989,-0.794891,-0.120201,-0.878175,1.107431,-0.240978,-1.025689,1.227165,0.163723,-0.211381,-0.318574,-0.950901,0.222012,0.311618,-0.060710,0.684189,-0.704242,-0.359202,-0.116299,1.164671,-0.068668,-0.087658,0.250805,0.138730
90,-0.872264,0.292142,-0.332305,-1.517947,-0.517023,-0.704164,-1.688790,-0.566122,-0.972685,-0.288554,-1.283736,-2.410341,-0.317203,-0.794891,-0.120201,-0.904817,-0.819684,-0.240978,-1.025689,-0.761360,-1.062101,-0.211381,-1.549046,-0.950901,0.182125,0.311618,-0.247797,-0.751918,-0.266546,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,0.250805,-1.367186
282,1.491770,-0.088804,-0.546407,0.651256,-0.517023,1.183071,1.120552,0.350052,1.009345,-0.288554,-0.355855,0.584836,0.391563,-0.794891,-0.120201,-0.383390,1.107431,-0.240978,0.789470,-0.761360,-1.062101,-0.211381,-0.318574,0.600289,0.303887,0.311618,0.715699,0.620362,0.231522,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-0.858816,0.891688
1397,0.309753,0.160276,-0.440508,-0.794879,2.178881,-1.697446,0.926804,-0.566122,-0.972685,-0.288554,0.841338,-0.269952,-0.578463,0.520029,-0.120201,-0.004688,-0.819684,-0.240978,-1.025689,1.227165,1.389547,-0.211381,0.911897,-0.950901,0.146436,-1.026506,-1.370316,-0.560437,-0.704242,2.095021,-0.116299,-0.270116,-0.068668,-0.087658,-0.488943,-0.614228
619,0.073350,0.658437,0.173041,1.374324,-0.517023,1.050634,0.878367,0.681199,0.936992,-0.288554,0.099033,0.967781,0.826134,0.991934,-0.120201,1.420674,1.107431,-0.240978,0.789470,1.227165,1.389547,-0.211381,2.142369,2.151479,0.293390,1.649742,1.290990,0.588449,-0.704242,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,0.620678,0.138730


In [365]:

'''Class for Box Kernel Regression'''
class BKR:
    
    def __init__(self, x, y, h):
        self.x = np.array(x)
        self.y = np.array(y)
        self.h = h
    
    '''Implement the Box Kernel'''
    def box_kernel(self, z, h):
        return 1 if z <= h else 0
    
    '''Calculate weights and return prediction'''
    def predict(self, X):

        kernels = np.array([self.box_kernel((np.linalg.norm(xi-X)),h) for xi in self.x])
        kernel_sum = np.sum(kernels)
        if kernel_sum == 0:
            return 0
        else:
            weights = np.array([len(self.x) * (kernel/kernel_sum) for kernel in kernels])
            return np.dot(weights.T, self.y)/len(self.x)


In [ ]:

'''Class for Gaussian Kernel Regression'''
class GKR:
    
    def __init__(self, x, y, sigma):
        self.x = np.array(x)
        self.y = np.array(y)
        self.sigma = sigma
    
    '''Implement the Gaussian Kernel'''
    def gaussian_kernel(self, z):
        return (1/np.sqrt(2*np.pi))*np.exp(-0.5*z**2)
    
    '''Calculate weights and return prediction'''
    def predict(self, X):
        kernels = np.array([self.gaussian_kernel((np.linalg.norm(xi-X))/self.sigma) for xi in self.x])
        weights = np.array([len(self.x) * (kernel/np.sum(kernels)) for kernel in kernels])
        return np.dot(weights.T, self.y)/len(self.x)

In [ ]:
sf=GKR(np.array(X_train),np.array(Y_train),1.35)
Y_pred = []
for input_x in np.array(X_validation):
    single_y = GKR.predict(sf,input_x)
    Y_pred.append(single_y)

Y_pred

[167608.56813559038,
 118277.69596822845,
 96546.47491672436,
 120723.25928536302,
 145726.0040626726,
 247699.90149928402,
 160727.85603109628,
 107339.64599424008,
 120069.57578017499,
 131698.7219122399,
 203608.6988956061,
 151907.28414381365,
 316639.14857871254,
 205860.11470068665,
 171436.35813022446,
 148688.1739861105,
 173732.576577041,
 185862.13958266622,
 206402.70758411352,
 309118.1298546501,
 249945.432977044,
 230203.54504000128,
 121175.8889392597,
 110818.63900974796,
 121424.78330441873,
 112546.85620170928,
 142485.15378746862,
 125079.72338775889,
 274002.46206344105,
 189958.3433912263,
 202117.55780946097,
 142287.66521433915,
 222648.80665681817,
 301897.23571330303,
 190044.83579219054,
 263573.9541491923,
 291517.5163979562,
 113896.56755040458,
 255171.19845663852,
 104169.07215888197,
 300782.6577288646,
 237915.29431265334,
 91908.95061946021,
 136880.30987114905,
 303262.70121399034,
 243203.23018245315,
 111481.80113527575,
 142614.60661988083,
 94879.0

In [376]:
# Compute RMSE for Box kernel
sf=BKR(np.array(X_train),np.array(Y_train),7)
Y_pred = []
for input_x in np.array(X_validation):
    single_y = BKR.predict(sf,input_x)
    Y_pred.append(single_y)

RMSE_h_best=compute_rmse(Y_train_validation,Y_pred)
print(f"Best RMSE for box kernel with h=7 is : {41614.235737}")

Best RMSE for box kernel with h=7 is : 41614.235737


In [ ]:
# Compute RMSE for Gaussian kernel
def compute_rmse(Y_train_validation,Y_pred):
    Y_train_validation=np.array(Y_train_validation)
    Y_pred=np.array(Y_pred)
    RMSE = np.sqrt(np.square(np.subtract(Y_train_validation,Y_pred)).mean())
    return RMSE

sf=GKR(np.array(X_train),np.array(Y_train),1.35)
Y_pred = []
for input_x in np.array(X_validation):
    single_y = GKR.predict(sf,input_x)
    Y_pred.append(single_y)

RMSE_sigma_best=compute_rmse(Y_train_validation,Y_pred)
print(f"Best RMSE for gaussian kernel with sigma=1.35 is : {RMSE_sigma_best}")

Best RMSE for gaussian kernel with sigma=1.35 is : 28045.722024


In [372]:
h_list=np.arange(0.5,10,0.5)
box_RMSE_list=[]
for h in h_list:
    sf=BKR(np.array(X_train),np.array(Y_train),h)
    Y_pred = []
    for input_x in np.array(X_validation):
        single_y = BKR.predict(sf,input_x)
        Y_pred.append(single_y)
    box_RMSE_list.append([h,compute_rmse(Y_train_validation,Y_pred)])


In [373]:
rmse_h_plot = pd.DataFrame({'h value':np.array(box_RMSE_list)[:,0], 'rmse':np.array(box_RMSE_list)[:,1]})
print(f"rmse_h_plot : \n{rmse_h_plot}")

rmse_h_plot : 
    h value           rmse
0       0.5  190514.020608
1       1.0  189596.192849
2       1.5  185510.380626
3       2.0  177114.124034
4       2.5  159236.539230
5       3.0  139470.586685
6       3.5  122994.877439
7       4.0   99602.780743
8       4.5   85519.175843
9       5.0   68423.327250
10      5.5   55398.515690
11      6.0   47675.421233
12      6.5   46470.631222
13      7.0   41614.235737
14      7.5   45476.271945
15      8.0   49376.686534
16      8.5   52727.280727
17      9.0   56017.902713
18      9.5   58833.743849


In [ ]:
h_list=np.arange(1,2,0.05)
RMSE_list=[]
Rect_RMSE_list=[]
for h in h_list:
    sf=GKR(np.array(X_train),np.array(Y_train),h)
    Y_pred = []
    for input_x in np.array(X_validation):
        single_y = GKR.predict(sf,input_x)
        Y_pred.append(single_y)
    RMSE_list.append([h,compute_rmse(Y_train_validation,Y_pred)])

In [ ]:
rmse_sigma_plot = pd.DataFrame({'sigma value':np.array(RMSE_list)[:,0], 'rmse':np.array(RMSE_list)[:,1]})
print(f"rmse_sigma_plot : \n{rmse_sigma_plot}")

rmse_sigma_plot : 
    sigma value          rmse
0          1.00  29168.251231
1          1.05  28829.836988
2          1.10  28551.315511
3          1.15  28332.314138
4          1.20  28172.554272
5          1.25  28071.983034
6          1.30  28030.187376
7          1.35  28045.722024
8          1.40  28115.860887
9          1.45  28236.916444
10         1.50  28404.915179
11         1.55  28616.255512
12         1.60  28868.058038
13         1.65  29158.134744
14         1.70  29484.697529
15         1.75  29846.005053
16         1.80  30240.107361
17         1.85  30664.747434
18         1.90  31117.389223
19         1.95  31595.304853


In [377]:
fig = px.line(rmse_h_plot, x="h value", y="rmse", title='rmse_h_plot for box kernel')
fig.show()

In [378]:
fig = px.line(rmse_sigma_plot, x="sigma value", y="rmse", title='rmse_sigma_plot for Gaussian kernel')
fig.show()

In [ ]:
# predict on test set
Y_test_pred = []
for input_x in np.array(test_copy_comm):
    single_y = GKR.predict(sf,input_x)
    Y_test_pred.append(single_y)

In [ ]:
testset_prediction = pd.DataFrame({'index':np.array(range(test_copy_comm.shape[0])), 'predicted value':np.array(Y_test_pred)})
fig = px.line(testset_prediction, x="index", y="predicted value", title='testset_prediction')
fig.show()

In [ ]:
trainset_actual_value = pd.DataFrame({'index':np.array(range(train_copy_comm.shape[0])), 'actual value':np.array(y)})
fig = px.line(trainset_actual_value, x="index", y="actual value", title='trainset_actual_value')
fig.show()